In [2]:
from utz import *

from tabula import read_pdf

In [35]:
year = 2020
county = 'Hudson'
dtype = 'Accidents'

In [14]:
rect = {
    "x1": 43.222500000000004,
    "x2": 564.1875,
    "y1": 91.4175,
    "y2": 750.0825,
}

pdf = '2017CrashTable.pdf'
tbls = read_pdf(pdf, area=[ rect[k] for k in [ 'y1', 'x1', 'y2', 'x2', ] ], pages='all',)
fields = pd.concat(tbls)
fields

,Field,Right\rJustify,From,To,Length,Comments / Notes
0,Year,NaN,1,4,4,NaN
1,County Code,NaN,5,6,2,NaN
2,Municipality Code,NaN,7,8,2,NaN
3,Department Case Number,NaN,9,31,23,NaN
4,Comma,NaN,32,32,1,NaN
...,...,...,...,...,...,...
15,Cell Phone In Use Flag,NaN,382,382,1,"Set to 'Y' if any driver, pedestrian or pedalc..."
16,Comma,NaN,383,383,1,NaN
17,Other Property Damage,NaN,384,463,80,NaN
18,Comma,NaN,464,464,1,NaN


In [32]:
def parse_row(f, idx):
    row = {}
    for fidx, field in enumerate(fields.to_dict('records')):
        fname, flen = field['Field'], field['Length']
        fval = f.read(flen)
        if not fval:
            if fidx:
                raise RuntimeError(f'row {idx} fidx {fidx} {fname}, empty read. {row}')
            else:
                return None
        fval = fval.strip()
        if fname != 'Comma':
            row[fname] = fval
    last = f.read(1)
    if last != '\n':
        raise RuntimeError(f'Row {idx}: expected newline, found "{last}", row {row}')
    return row

In [33]:
%%time
rows = []
idx = 0
name = f'{year}/{county}{year}{dtype}'
with open(f'{name}.txt', 'r') as f:
    while True:
        row = parse_row(f, idx=idx)
        if row:
            rows.append(row)
            idx += 1
        else:
            break

df = DF(rows)
df

CPU times: user 4.21 s, sys: 69.3 ms, total: 4.28 s
Wall time: 4.29 s


,Year,County Code,Municipality Code,Department Case Number,County Name,Municipality Name,Crash Date,Crash Day Of Week,Crash Time,Police Dept Code,Police Department,Police Station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,Severity,Intersection,Alcohol Involved,HazMat Involved,Crash Type Code,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,SRI (Std Rte Identifier),MilePost,Road System,Road Character,Road Horizontal Alignment,Road Grade,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Directn From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,First Harmful Event,Latitude,Longitude,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.
0,2020,09,01,20-004377,HUDSON,BAYONNE CITY,02/25/2020,TU,1319,01,JERSEY CITY PD,NORTH,0,1,0,0,I,I,N,N,14,1,JAMES AVE,S,,,,,07,,01,04,02,02,01,02,04,01,,AT,,ST. PAULS AVE,,,,25,25,26,,,N,,3101
1,2020,09,01,20-013614,HUDSON,BAYONNE CITY,08/04/2020,TU,2211,01,JERSEY CITY PD,SOUTH,0,1,0,0,I,I,N,N,02,3,ROUTE 501,W,501,,00000501__,26.65,05,,01,04,02,02,07,02,04,01,,AT,,W 53RD ST,,,,25,25,26,,,N,,3106
2,2020,09,01,20-032209,HUDSON,BAYONNE CITY,06/13/2020,SA,1233,01,BAYONNE PD,HQ,0,0,0,0,P,B,N,N,06,2,W 13TH ST **,W,,,09011151__,.22,07,,01,04,02,01,01,01,05,01,25,FE,W,AVENUE C,,,,25,25,26,,,N,,#8049
3,2020,09,01,20-038205,HUDSON,BAYONNE CITY,07/15/2020,WE,1138,01,BAYONNE PD,HQ,0,0,0,0,P,I,N,N,03,2,AVENUE C,E,,,09011543__,.11,07,,01,04,02,01,01,01,05,01,,AT,,W 2ND ST,,,,25,25,26,,,N,,353
4,2020,09,01,20-038566,HUDSON,BAYONNE CITY,07/17/2020,FR,0636,01,BAYONNE PD,HQ,0,1,0,0,I,B,N,N,02,2,2ND ST E **,E,,,09011564__,.08,07,,01,04,02,01,01,05,05,01,100,FE,E,LEXINGTON AVE,,,,25,25,26,,,N,,353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13094,2020,09,12,SO-20-006766,HUDSON,WEST NEW YORK TOWN,02/17/2020,MO,0950,05,HUDSON CO SHERIFFS DEPT,PATROL,0,0,0,0,P,I,N,N,02,2,ROUTE 501,N,501,,00000501__,35.35,05,,03,04,02,01,01,01,04,01,,AT,,56TH ST,,,,25,25,26,,,N,,1112
13095,2020,09,12,SO-20-012195,HUDSON,WEST NEW YORK TOWN,03/16/2020,MO,1510,05,HUDSON CO SHERIFFS DEPT,PATROL,0,0,0,0,P,I,N,N,04,2,ROUTE 501,N,501,,00000501__,35.65,05,,01,04,02,01,01,01,04,01,,AT,,61ST ST,,,,25,25,26,,,N,,1112
13096,2020,09,12,SO-20-045732,HUDSON,WEST NEW YORK TOWN,10/08/2020,TH,1501,05,HUDSON CO SHERIFFS DEPT,PATROL BUREAU,0,0,0,0,P,I,N,N,03,2,ROUTE 501,N,501,,00000501__,35.65,05,,01,04,02,01,01,01,04,01,,AT,,61ST ST,,,,25,25,26,,,N,,I-34
13097,2020,09,12,SO-20-050243,HUDSON,WEST NEW YORK TOWN,10/29/2020,TH,1308,05,HUDSON CO SHERIFFS DEPT,PATROL,0,0,0,0,P,I,N,N,02,2,ROUTE 501,N,501,,00000501__,35.14,05,,01,04,02,02,01,02,04,01,,AT,,52ND ST,,,,25,25,26,,,N,,I-38


In [46]:
df.to_parquet(f'{name}.pqt', index=None)

In [44]:
df.Severity.value_counts()

P    10706
I     2369
F       24
Name: Severity, dtype: int64

In [41]:
df[['Crash Location', 'Cross Street Name']].replace('', nan).isna().sum(0)

Crash Location          0
Cross Street Name    1859
dtype: int64

In [43]:
df['Total Killed'].astype(int).sum()

24

In [40]:
df[['Total Killed', 'Total Injured']].value_counts()

Total Killed  Total Injured
0             0                10706
              1                 1870
              2                  363
              3                   89
              4                   29
1             0                   22
0             6                    7
              5                    6
              7                    2
              8                    2
1             2                    2
0             10                   1
dtype: int64

In [39]:
(~df[['Latitude', 'Longitude']].replace('', nan).isna()).sum(axis=0)

Latitude     1399
Longitude    1399
dtype: int64

In [37]:
df['HazMat Involved'].value_counts()

N    13096
Y        3
Name: HazMat Involved, dtype: int64

In [36]:
df['Alcohol Involved'].value_counts()

N    12778
Y      321
Name: Alcohol Involved, dtype: int64

In [34]:
df[df['Other Property Damage'].str.contains('\n')]

,Year,County Code,Municipality Code,Department Case Number,County Name,Municipality Name,Crash Date,Crash Day Of Week,Crash Time,Police Dept Code,Police Department,Police Station,Total Killed,Total Injured,Pedestrians Killed,Pedestrians Injured,Severity,Intersection,Alcohol Involved,HazMat Involved,Crash Type Code,Total Vehicles Involved,Crash Location,Location Direction,Route,Route Suffix,SRI (Std Rte Identifier),MilePost,Road System,Road Character,Road Horizontal Alignment,Road Grade,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Directn From Cross Street,Cross Street Name,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,First Harmful Event,Latitude,Longitude,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.
3490,2020,09,06,20-006182,HUDSON,JERSEY CITY,03/21/2020,SA,0251,01,JERSEY CITY PD,NORTH,0,0,0,0,P,I,N,N,02,2,SUMMIT AVE,,,,09061700__,.08,07,,01,04,02,01,06,01,05,01,,AT,,SHERMAN PL,,,,25,25,26,,,N,907 SUMMIT AVE FRONT FENCE SIGN POST\n909 SUMM...,3043
10413,2020,09,08,D030-2020-00004A,HUDSON,NORTH BERGEN TWP,01/01/2020,WE,2128,02,NEW JERSEY STATE POLICE,NEWARK - SQUAD,0,0,0,0,P,B,Y,N,11,1,I-95 N.J. TURNPIKE,N,95,,00000095__,69.17,03,,01,04,02,01,01,01,01,01,,,,,,,,55,,46,40.811240,74.029340,N,GUARDRAIL \nNJTP;PO BOX 5042;WOODBRIDGE NJ 07095,8063


In [10]:
h20 = read_csv('2020/Hudson2020Accidents.txt', header=None)
h20

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,2020090120-004377,HUDSON,BAYONNE CITY,02/25/2020,TU,1319,1.0,JERSEY CITY PD,NORTH,0.0,1.0,0.0,0.0,I,I,N,N,14,1.0,JAMES AVE ...,S,,,,,7.0,,01,04,02,02,01,02,04,01,,AT,,ST. PAULS AVE,,,,25,25,26,,,N,...,3101
1,2020090120-013614,HUDSON,BAYONNE CITY,08/04/2020,TU,2211,1.0,JERSEY CITY PD,SOUTH,0.0,1.0,0.0,0.0,I,I,N,N,02,3.0,ROUTE 501 ...,W,501,,00000501__,26.65,5.0,,01,04,02,02,07,02,04,01,,AT,,W 53RD ST,,,,25,25,26,,,N,...,3106
2,2020090120-032209,HUDSON,BAYONNE CITY,06/13/2020,SA,1233,1.0,BAYONNE PD,HQ,0.0,0.0,0.0,0.0,P,B,N,N,06,2.0,W 13TH ST ** ...,W,,,09011151__,.22,7.0,,01,04,02,01,01,01,05,01,25,FE,W,AVENUE C,,,,25,25,26,,,N,...,#8049
3,2020090120-038205,HUDSON,BAYONNE CITY,07/15/2020,WE,1138,1.0,BAYONNE PD,HQ,0.0,0.0,0.0,0.0,P,I,N,N,03,2.0,AVENUE C ...,E,,,09011543__,.11,7.0,,01,04,02,01,01,01,05,01,,AT,,W 2ND ST,,,,25,25,26,,,N,...,353
4,2020090120-038566,HUDSON,BAYONNE CITY,07/17/2020,FR,0636,1.0,BAYONNE PD,HQ,0.0,1.0,0.0,0.0,I,B,N,N,02,2.0,2ND ST E ** ...,E,,,09011564__,.08,7.0,,01,04,02,01,01,05,05,01,100,FE,E,LEXINGTON AVE,,,,25,25,26,,,N,...,353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13097,20200912SO-20-006766,HUDSON,WEST NEW YORK TOWN,02/17/2020,MO,0950,5.0,HUDSON CO SHERIFFS DEPT,PATROL,0.0,0.0,0.0,0.0,P,I,N,N,02,2.0,ROUTE 501 ...,N,501,,00000501__,35.35,5.0,,03,04,02,01,01,01,04,01,,AT,,56TH ST,,,,25,25,26,,,N,...,1112
13098,20200912SO-20-012195,HUDSON,WEST NEW YORK TOWN,03/16/2020,MO,1510,5.0,HUDSON CO SHERIFFS DEPT,PATROL,0.0,0.0,0.0,0.0,P,I,N,N,04,2.0,ROUTE 501 ...,N,501,,00000501__,35.65,5.0,,01,04,02,01,01,01,04,01,,AT,,61ST ST,,,,25,25,26,,,N,...,1112
13099,20200912SO-20-045732,HUDSON,WEST NEW YORK TOWN,10/08/2020,TH,1501,5.0,HUDSON CO SHERIFFS DEPT,PATROL BUREAU,0.0,0.0,0.0,0.0,P,I,N,N,03,2.0,ROUTE 501 ...,N,501,,00000501__,35.65,5.0,,01,04,02,01,01,01,04,01,,AT,,61ST ST,,,,25,25,26,,,N,...,I-34
13100,20200912SO-20-050243,HUDSON,WEST NEW YORK TOWN,10/29/2020,TH,1308,5.0,HUDSON CO SHERIFFS DEPT,PATROL,0.0,0.0,0.0,0.0,P,I,N,N,02,2.0,ROUTE 501 ...,N,501,,00000501__,35.14,5.0,,01,04,02,02,01,02,04,01,,AT,,52ND ST,,,,25,25,26,,,N,...,I-38


In [12]:
h20[0].apply(len).value_counts()

31    13099
43        1
4         1
69        1
Name: 0, dtype: int64

In [14]:
h20[0].str.extract('(?P<year>....)(?P<ccode>..)(?P<mcode>..)(?P<case_id>.{23})')

,year,ccode,mcode,case_id
0,2020,09,01,20-004377
1,2020,09,01,20-013614
2,2020,09,01,20-032209
3,2020,09,01,20-038205
4,2020,09,01,20-038566
...,...,...,...,...
13097,2020,09,12,SO-20-006766
13098,2020,09,12,SO-20-012195
13099,2020,09,12,SO-20-045732
13100,2020,09,12,SO-20-050243


In [19]:
h20[h.year.isna()]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
7163,,1014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
h[1].apply(len).value_counts()

12    13099
5         3
Name: 1, dtype: int64

In [16]:
#h20 = h20.drop(columns='year')
h = sxs(h20.drop(columns=[0]), h20[0].str.extract('(?P<year>....)(?P<ccode>..)(?P<mcode>..)(?P<case_id>.{23})'))
h

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,year,ccode,mcode,case_id
0,HUDSON,BAYONNE CITY,02/25/2020,TU,1319,1.0,JERSEY CITY PD,NORTH,0.0,1.0,0.0,0.0,I,I,N,N,14,1.0,JAMES AVE ...,S,,,,,7.0,,01,04,02,02,01,02,04,01,,AT,,ST. PAULS AVE,,,,25,25,26,,,N,...,3101,2020,09,01,20-004377
1,HUDSON,BAYONNE CITY,08/04/2020,TU,2211,1.0,JERSEY CITY PD,SOUTH,0.0,1.0,0.0,0.0,I,I,N,N,02,3.0,ROUTE 501 ...,W,501,,00000501__,26.65,5.0,,01,04,02,02,07,02,04,01,,AT,,W 53RD ST,,,,25,25,26,,,N,...,3106,2020,09,01,20-013614
2,HUDSON,BAYONNE CITY,06/13/2020,SA,1233,1.0,BAYONNE PD,HQ,0.0,0.0,0.0,0.0,P,B,N,N,06,2.0,W 13TH ST ** ...,W,,,09011151__,.22,7.0,,01,04,02,01,01,01,05,01,25,FE,W,AVENUE C,,,,25,25,26,,,N,...,#8049,2020,09,01,20-032209
3,HUDSON,BAYONNE CITY,07/15/2020,WE,1138,1.0,BAYONNE PD,HQ,0.0,0.0,0.0,0.0,P,I,N,N,03,2.0,AVENUE C ...,E,,,09011543__,.11,7.0,,01,04,02,01,01,01,05,01,,AT,,W 2ND ST,,,,25,25,26,,,N,...,353,2020,09,01,20-038205
4,HUDSON,BAYONNE CITY,07/17/2020,FR,0636,1.0,BAYONNE PD,HQ,0.0,1.0,0.0,0.0,I,B,N,N,02,2.0,2ND ST E ** ...,E,,,09011564__,.08,7.0,,01,04,02,01,01,05,05,01,100,FE,E,LEXINGTON AVE,,,,25,25,26,,,N,...,353,2020,09,01,20-038566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13097,HUDSON,WEST NEW YORK TOWN,02/17/2020,MO,0950,5.0,HUDSON CO SHERIFFS DEPT,PATROL,0.0,0.0,0.0,0.0,P,I,N,N,02,2.0,ROUTE 501 ...,N,501,,00000501__,35.35,5.0,,03,04,02,01,01,01,04,01,,AT,,56TH ST,,,,25,25,26,,,N,...,1112,2020,09,12,SO-20-006766
13098,HUDSON,WEST NEW YORK TOWN,03/16/2020,MO,1510,5.0,HUDSON CO SHERIFFS DEPT,PATROL,0.0,0.0,0.0,0.0,P,I,N,N,04,2.0,ROUTE 501 ...,N,501,,00000501__,35.65,5.0,,01,04,02,01,01,01,04,01,,AT,,61ST ST,,,,25,25,26,,,N,...,1112,2020,09,12,SO-20-012195
13099,HUDSON,WEST NEW YORK TOWN,10/08/2020,TH,1501,5.0,HUDSON CO SHERIFFS DEPT,PATROL BUREAU,0.0,0.0,0.0,0.0,P,I,N,N,03,2.0,ROUTE 501 ...,N,501,,00000501__,35.65,5.0,,01,04,02,01,01,01,04,01,,AT,,61ST ST,,,,25,25,26,,,N,...,I-34,2020,09,12,SO-20-045732
13100,HUDSON,WEST NEW YORK TOWN,10/29/2020,TH,1308,5.0,HUDSON CO SHERIFFS DEPT,PATROL,0.0,0.0,0.0,0.0,P,I,N,N,02,2.0,ROUTE 501 ...,N,501,,00000501__,35.14,5.0,,01,04,02,02,01,02,04,01,,AT,,52ND ST,,,,25,25,26,,,N,...,I-38,2020,09,12,SO-20-050243
